# Classfication Network Architecture

I am unable to get flowers classification network to get a validation accuracy > 48%. Try out different techniques including the same architecture I used to train CIFAR10 images

@date: 06-Aug-2020 | @author: katnoria

In [66]:
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPool2D
from tensorflow.keras.layers import BatchNormalization, Input, GlobalAveragePooling2D
from tensorflow.keras import Model

In [2]:
def version_info(cls):
    print(f"{cls.__name__}: {cls.__version__}")

In [3]:
print("Version Used in this Notebook:")
version_info(tf)
version_info(tfds)

Version Used in this Notebook:
tensorflow: 2.3.0
tensorflow_datasets: 3.2.1


In [4]:
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  2


In [5]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  # Restrict TensorFlow to only use the first GPU
  try:
    tf.config.experimental.set_visible_devices(gpus[1], 'GPU')
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
  except RuntimeError as e:
    # Visible devices must be set before GPUs have been initialized
    print(e)

2 Physical GPUs, 1 Logical GPU


# Dataset

Tensorflow Datasets already provides this dataset in a format that we can use out of the box.

In [18]:
# Load the dataset
(ds_train, ds_test), metadata = tfds.load(
    'cifar10', split=['train', 'test'], shuffle_files=True, 
    with_info=True, as_supervised=True
)

In [19]:
len(ds_train), len(ds_test), metadata.features['label'].num_classes

(50000, 10000, 10)

Use the built in function to visualise the dataset 

TypeError: show_examples() missing 1 required positional argument: 'ds_info'

In [24]:
# Review metadata
# See https://www.tensorflow.org/datasets/overview
metadata.features

FeaturesDict({
    'id': Text(shape=(), dtype=tf.string),
    'image': Image(shape=(32, 32, 3), dtype=tf.uint8),
    'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=10),
})

In [25]:
NUM_CLASSES = metadata.features["label"].num_classes

num_train_examples = len(ds_train)
num_test_examples = len(ds_test)
print(f"Training dataset size: {num_train_examples}")
print(f"Test dataset size: {num_test_examples}")

Training dataset size: 50000
Test dataset size: 10000


### Training Pipeline

In [37]:
IMG_SIZE = 32
BATCH_SIZE = 256


def preprocess_image(image, label):
    image = tf.image.convert_image_dtype(image, tf.float32)
#     image = tf.image.resize(image, (IMG_SIZE, IMG_SIZE))
    return image / 255., label

In [38]:
train_ds = ds_train.map(preprocess_image, num_parallel_calls=tf.data.experimental.AUTOTUNE) \
    .cache() \
    .shuffle(num_train_examples).batch(BATCH_SIZE, drop_remainder=True) \
    .prefetch(tf.data.experimental.AUTOTUNE) 

In [39]:
example = ds_train.take(1)

In [40]:
example

<TakeDataset shapes: ((32, 32, 3), ()), types: (tf.uint8, tf.int64)>

### Evaluation Pipeline

In [41]:
test_ds = ds_test.map(preprocess_image, num_parallel_calls=tf.data.experimental.AUTOTUNE) \
    .cache() \
    .batch(BATCH_SIZE, drop_remainder=True) \
    .prefetch(tf.data.experimental.AUTOTUNE) 

# Build Model

We now build a simple convolution neural network

In [89]:
class FlowersModel(Model):
    def __init__(self):
        super(FlowersModel, self).__init__()
        self.conv1 = Conv2D(32, 3, padding='same')
        self.bn1 = BatchNormalization()
        self.conv2 = Conv2D(64, 3, padding='same')
        self.pool1 = MaxPool2D(3, 2)
        self.bn2 = BatchNormalization()        
        self.pool2 = MaxPool2D(3, 2)        
        self.flatten = Flatten()
        self.dense1 = Dense(128, activation='relu')
        self.gap = GlobalAveragePooling2D()
        self.dense2 = Dense(NUM_CLASSES)
        
    def call(self, x, training=False):
        x = self.conv1(x)
#         x = self.bn1(x, training=training)
        x = self.pool1(tf.nn.relu(x))
        
        x = self.conv2(x)
#         x = self.bn2(x, training=training)
        x = tf.nn.relu(x)
        x = self.pool2(x)
#         x = self.gap(x)
        x = self.flatten(x)
        x = self.dense1(x)
        if training:
            x = tf.nn.dropout(x, rate=0.2)
        out = self.dense2(x)
        return out

In [90]:
model = FlowersModel()

Create the loss function and optimizer

In [91]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
optimizer = tf.keras.optimizers.Adam()

We are going to measure the train and test accuracy of the model

In [92]:
# Train
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_acc = tf.keras.metrics.SparseCategoricalAccuracy(name='train_acc')

# Test
test_loss = tf.keras.metrics.Mean(name='test_loss')
test_acc = tf.keras.metrics.SparseCategoricalAccuracy(name='test_acc')

We now train the model

In [93]:
# Training step
@tf.function
def train_step(images, labels):
    with tf.GradientTape() as tape:
        predictions = model(images, training=True)
        loss = loss_fn(labels, predictions)
    # collect the gradients and apply
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    # loss & acc
    train_loss(loss)
    train_acc(labels, predictions)
    
    
# Test step    
@tf.function
def test_step(images, labels):
    predictions = model(images, training=False)
    loss = loss_fn(labels, predictions)
    # loss & acc
    test_loss(loss)
    test_acc(labels, predictions)

# Train the model

Now, its time to train the model for N epochs

In [95]:
num_epochs = 100
print_every = 10

for epoch in range(num_epochs):
    train_loss.reset_states()
    train_acc.reset_states()
    test_loss.reset_states()
    test_acc.reset_states()
    
    for images, labels in train_ds:
        train_step(images, labels)
        
    for tst_images, tst_labels in test_ds:
        test_step(tst_images, tst_labels)
        
    if epoch % print_every == 0:
        print(f"Epoch {epoch+1}: loss={train_loss.result():.4f}, accuracy: {train_acc.result():.4f} :: test loss={test_loss.result():.4f}, test accuracy: {test_acc.result():.4f}")

Epoch 1: loss=1.4669, accuracy: 0.4736 :: test loss=1.4060, test accuracy: 0.4976
Epoch 11: loss=1.3795, accuracy: 0.5052 :: test loss=1.3156, test accuracy: 0.5318
Epoch 21: loss=1.3261, accuracy: 0.5241 :: test loss=1.2891, test accuracy: 0.5435
Epoch 31: loss=1.2658, accuracy: 0.5456 :: test loss=1.2228, test accuracy: 0.5704
Epoch 41: loss=1.2273, accuracy: 0.5628 :: test loss=1.1942, test accuracy: 0.5824
Epoch 51: loss=1.1812, accuracy: 0.5797 :: test loss=1.1599, test accuracy: 0.5907
Epoch 61: loss=1.1516, accuracy: 0.5892 :: test loss=1.1262, test accuracy: 0.6005
Epoch 71: loss=1.1177, accuracy: 0.6030 :: test loss=1.1111, test accuracy: 0.6060
Epoch 81: loss=1.0945, accuracy: 0.6098 :: test loss=1.0980, test accuracy: 0.6127
Epoch 91: loss=1.0611, accuracy: 0.6198 :: test loss=1.0816, test accuracy: 0.6169


# Use Keras fit Method

In [88]:
model = FlowersModel()
model.compile(
    loss=loss_fn, 
    optimizer=tf.keras.optimizers.Adam(0.001),
    metrics=['accuracy'],    
)
model.fit(
    train_ds,
    epochs=10,
    validation_data=test_ds,
)

Epoch 1/10
  1/195 [..............................] - ETA: 0s - loss: 2.4158 - accuracy: 0.0820WARNING:tensorflow:Callbacks method `on_train_batch_end` is slow compared to the batch time (batch time: 0.0033s vs `on_train_batch_end` time: 0.0052s). Check your callbacks.


195/195 [==============================] - 2s 10ms/step - loss: 1.4477 - accuracy: 0.4803 - val_loss: 2.3369 - val_accuracy: 0.1000
Epoch 2/10
195/195 [==============================] - 2s 9ms/step - loss: 1.0864 - accuracy: 0.6147 - val_loss: 6.5635 - val_accuracy: 0.1000
Epoch 3/10
195/195 [==============================] - 2s 9ms/step - loss: 0.9503 - accuracy: 0.6647 - val_loss: 9.6181 - val_accuracy: 0.1012
Epoch 4/10
195/195 [==============================] - 2s 9ms/step - loss: 0.8772 - accuracy: 0.6907 - val_loss: 34.9941 - val_accuracy: 0.1000
Epoch 5/10
195/195 [==============================] - 2s 9ms/step - loss: 0.8220 - accuracy: 0.7115 - val_loss: 66.8259 - val_accuracy: 0.1033
Epoch 6/10
195/195 [==============================] - 2s 10ms/step - loss: 0.7678 - accuracy: 0.7300 - val_loss: 61.1285 - val_accuracy: 0.1003
Epoch 7/10
195/195 [==============================] - 2s 9ms/step - loss: 0.7311 - accuracy: 0.7434 - val_loss: 137.5869 - val_accuracy: 0.0999
Epoch 8/10